<a href="https://colab.research.google.com/github/krishshhah/LuxSwap/blob/main/LuxSwap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

In [2]:
path = kagglehub.dataset_download("dnepozitek/polyvore-outfits")

100%|██████████| 6.21G/6.21G [01:14<00:00, 89.0MB/s]

Extracting files...


In [4]:
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/dnepozitek/polyvore-outfits/versions/1


In [3]:
import json

with open(path + '/polyvore_outfits/disjoint/train.json', 'r') as file:
    data = json.load(file)

result = [
    [item["item_id"] for item in entry["items"][:2]]
    for entry in data
]

del data
print(result)

[['132621870', '153967122'], ['162715806', '171888747'], ['180028994', '182218570'], ['198643069', '206048471'], ['200139640', '156489567'], ['208870650', '207173723'], ['199376553', '203551771'], ['210942072', '212839321'], ['192511179', '187317072'], ['173097736', '172685140'], ['213007746', '212992927'], ['209966118', '70625358'], ['158857944', '88909845'], ['133932837', '161950492'], ['204211869', '204273384'], ['205039314', '205037505'], ['209609138', '210544460'], ['114654675', '146875636'], ['117267686', '128309696'], ['204925548', '213776990'], ['148741825', '127890350'], ['110743844', '97417141'], ['170205623', '160420622'], ['101993130', '107333685'], ['191328899', '191201889'], ['75817384', '77419985'], ['203989758', '203934247'], ['211016756', '210822844'], ['198348970', '151371943'], ['148428284', '149211113'], ['173531714', '163923975'], ['153347062', '21327665'], ['191321013', '204224738'], ['204165631', '209074952'], ['70510470', '70588472'], ['204084161', '200951415'],

In [4]:
for x in range(len(result) - 1):
  result[x].append(result[x+1][0])

result = result[:500]

print(len(result))

500


In [5]:
from PIL import Image
import numpy as np

def preprocess(path):
# Function to preprocess the image
    image = Image.open(path)
    # Opens the image at the specified path
    image = image.resize((224, 224))
    # Resizing the image to 300x300 pixels
    image = np.array(image, dtype=np.float32)
    # Converting the image into a tensor
    image = image / 255.0
    # Normalising the image [0-1]
    return image

anchor = []
positive = []
negative = []

base_path = '/root/.cache/kagglehub/datasets/dnepozitek/polyvore-outfits/versions/1/polyvore_outfits/images/'
for triplet in result:
  anchor.append(preprocess(base_path + triplet[0] + '.jpg'))
  positive.append(preprocess(base_path + triplet[1] + '.jpg'))
  negative.append(preprocess(base_path + triplet[2] + '.jpg'))

In [6]:
!pip install tensorflow

In [7]:
import tensorflow as tf

anchor = tf.convert_to_tensor(anchor)
positive = tf.convert_to_tensor(positive)
negative = tf.convert_to_tensor(negative)

image_triplet = tf.concat([anchor, positive, negative], axis=1)

print(anchor.shape)

(500, 224, 224, 3)


In [8]:
import tensorflow as tf

class LuxSwap(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.model = tf.keras.Sequential([])
        self.model.add(tf.keras.applications.ResNet50(include_top=False, weights='imagenet', pooling='avg'))
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(units=512, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=256, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=32))

    def call(self, tensor):
      input_anchor, input_positive, input_negative = tf.split(tensor, num_or_size_splits=3, axis=1)
      embedding_anchor = self.model(input_anchor)
      embedding_positive = self.model(input_positive)
      embedding_negative = self.model(input_negative)

      return tf.concat([embedding_anchor, embedding_positive, embedding_negative], axis=1)

In [9]:
def triplet_loss(y_true, y_pred, margin=0.4):

    anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)

    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)

    loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
    return tf.reduce_mean(loss)

In [10]:
# Create the triplet model
model = LuxSwap()

# Compile with the custom triplet loss
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=triplet_loss
)

model.fit(
  x=image_triplet,
  y=tf.zeros([500]),
  batch_size=32,
  epochs=10
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 124s 3s/step - loss: 0.4024
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 66s 658ms/step - loss: 0.3777
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 654ms/step - loss: 0.3868
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 655ms/step - loss: 0.3948
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 655ms/step - loss: 0.3817
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 653ms/step - loss: 0.3848
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 658ms/step - loss: 0.3828
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 662ms/step - loss: 0.3540
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 658ms/step - loss: 0.3480
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 657ms/step - loss: 0.3701
